In [57]:
import pandas as pd


In [58]:
data = pd.read_csv("CP_count_Hofmarcher.csv")
data

,INCHIKEY,1,2,3,4,5,6,7,8,9,...,Cells_Neighbors_SecondClosestObjectNumber_5,Cells_Neighbors_SecondClosestObjectNumber_Adjacent,Cells_Parent_Nuclei,Cytoplasm_Number_Object_Number,Cytoplasm_Parent_Cells,Cytoplasm_Parent_Nuclei,Nuclei_Neighbors_FirstClosestObjectNumber_1,Nuclei_Neighbors_SecondClosestObjectNumber_1,Nuclei_Number_Object_Number,InChIKey
0,AACRWZVDRSTLKY-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.054688,-0.054688,0.859375,0.859375,0.859375,0.859375,1.445312,1.007812,0.859375,AACRWZVDRSTLKY-UHFFFAOYSA-N
1,AACUKVXTFOXDGE-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.171875,-1.171875,-1.148438,-1.148438,-1.148438,-1.148438,-1.101562,-1.453125,-1.148438,AACUKVXTFOXDGE-UHFFFAOYSA-N
2,AADCDMQTJNYOSS-LBPRGKRZSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.425781,-0.425781,0.453125,0.453125,0.453125,0.453125,-0.117188,-0.531250,0.453125,AADCDMQTJNYOSS-LBPRGKRZSA-N
3,AADORYZVGJDNSZ-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.123188,1.123188,0.521739,0.521739,0.521739,0.521739,1.536232,0.601449,0.521739,AADORYZVGJDNSZ-UHFFFAOYSA-N
4,AAEVYOVXGOFMJO-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-6.882812,-6.882812,-7.218750,-7.218750,-7.218750,-7.218750,-7.679688,-7.773438,-7.218750,AAEVYOVXGOFMJO-UHFFFAOYSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10568,ZZUCJGSOKDNIEZ-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.609375,0.609375,0.679688,0.679688,0.679688,0.679688,0.757812,0.757812,0.679688,ZZUCJGSOKDNIEZ-UHFFFAOYSA-N
10569,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-10.242188,-10.242188,-10.746094,-10.746094,-10.746094,-10.746094,-11.167969,-10.281250,-10.746094,ZZUFCTLCJUWOSV-UHFFFAOYSA-N
10570,ZZUZYEMRHCMVTB-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-16.171875,-16.171875,-16.304688,-16.304688,-16.304688,-16.304688,-16.335938,-15.152344,-16.304688,ZZUZYEMRHCMVTB-UHFFFAOYSA-N
10571,ZZVUWRFHKOJYTH-UHFFFAOYSA-N,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.640625,-0.640625,-1.664062,-1.664062,-1.664062,-1.664062,-1.257812,-0.289062,-1.664062,ZZVUWRFHKOJYTH-UHFFFAOYSA-N


In [59]:
assays_list = data.columns.to_list()[1:-13]
assays_list

train_cols = ['Cells_Number_Object_Number']

In [60]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import logging


cp_path = "CP_count_Hofmarcher.csv"
cp_data = pd.read_csv(cp_path)

def merge_data(split_data, cp_data):
    # Assuming 'inchikey' is the column to merge on
    return pd.merge(split_data, cp_data, on='INCHIKEY', how='inner')

def load_and_preprocess_data(fold):
    train_path = f"data/datasplit{fold}-train.csv"
    val_path = f"data/datasplit{fold}-val.csv"
    test_path = f"data/datasplit{fold}-test.csv"

    # Load CP data
    cp_data = pd.read_csv(cp_path)
    
    # Load data splits
    train_df= pd.read_csv(train_path)
    val_df = pd.read_csv(val_path)
    test_df = pd.read_csv(test_path)
    
    # Merge the datasets
    train_merged = merge_data(train_df, cp_data)
    val_merged = merge_data(val_df, cp_data)
    test = merge_data(test_df, cp_data)
    
    train = pd.concat([train_merged,val_merged]).reset_index(drop=True)
    
    return train, test



In [93]:
import numpy as np
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

missing_value_indicator = -1

# Initialize a list to store AUC scores for all folds
all_auc_scores = []

# Initialize an empty list to hold models for each fold
models_per_fold = []

# Define the fold indices (adjust based on your data split)
folds = ["1", "2", "3"]

best_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)

# Loop through each fold, leaving it out as the test set and training on the others
for fold in tqdm(folds):
    
    if(best_clf):      
        del(best_clf)
        
    # Initialize lists to store data for the current fold's training set
    X_train_combined = pd.DataFrame()
    y_train_combined = pd.Series()

    #Train on all folds except the current one
    train_data, _ = load_and_preprocess_data(fold)

    # Loop through all assays and collect data
    for assay in assays_list:
        train_data_assay = train_data[train_data[assay]!=-1].reset_index(drop=True)[[assay, "Cells_Number_Object_Number"]]

        # Append to combined training data
        X_train_combined = pd.concat([X_train_combined, train_data_assay[["Cells_Number_Object_Number"]]], axis=0)
        y_train_combined = pd.concat([y_train_combined, train_data_assay[assay].astype(int)], axis=0)
            
    print(len(X_train_combined))
    print(len(y_train_combined))

    # Calculate class weights (balanced)
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_combined), y=y_train_combined)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(class_weight_dict)
    
    # Calculate scale_pos_weight based on class weights
    scale_pos_weight = class_weight_dict[0] / class_weight_dict[1]
    print("scale_pos_weight: ", scale_pos_weight)
        
    # Define the parameter grid with additional hyperparameters and wider ranges
    param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200],
    'scale_pos_weight': [scale_pos_weight],
    'n_jobs': [-1]
    }

    # Initialize the XGBoost classifier
    xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)

    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=xgb_clf,
        param_distributions=param_grid,
        n_iter=10,  # Number of parameter settings that are sampled
        scoring='f1',  # Metric for imbalanced dataset (you can also try 'roc_auc' or 'average_precision')
        cv=stratified_kfold,  # 5-fold cross-validation
        n_jobs=-1,  # Use all processors
        verbose=1,
        random_state=42
    )

    # Fit the model on training data
    random_search.fit(X_train_combined, y_train_combined)

    # Best estimator
    best_clf = random_search.best_estimator_
    print(f"Best parameters: {random_search.best_params_}")

    # Train the model on the combined data from other folds
    # clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)
    best_clf.fit(X_train_combined, y_train_combined)
       
    # Store the trained model
    models_per_fold.append(best_clf)

# Predict for each individual fold using the corresponding model
for idx, fold in tqdm(enumerate(folds)):
    
    # Load and preprocess data for the current test fold
    _, test_data = load_and_preprocess_data(fold)
    
    # Get the trained model for this fold (trained on other folds)
    clf = models_per_fold[idx]
    
    # Loop through each assay in the test set for the current fold
    for assay in tqdm(assays_list):
        test_data_assay = test_data[test_data[assay]!=-1].reset_index(drop=True)[[assay, "Cells_Number_Object_Number"]]
        
        # Separate features and target variable
        X_test = test_data_assay[["Cells_Number_Object_Number"]]
        y_test = test_data_assay[assay].astype(int)

        if len(y_test) == 0:
            print(f"Skipping assay {assay} in fold {fold} due to no test data")
            continue
        
        # Make predictions using the model for the current fold
        y_pred = clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)[:, 1]

        # Calculate AUC score
        auc_score = np.nan    
        try:
            auc_score = roc_auc_score(y_test, y_pred_proba)
        except:
            continue
        print(auc_score)

        # Store the AUC score along with fold and assay information
        all_auc_scores.append({"Fold": fold, "Task": assay, "AUC": auc_score})

# Convert the AUC results to a DataFrame for convenience
auc_df = pd.DataFrame(all_auc_scores)

# Save the DataFrame to a CSV file
auc_df.to_csv("auc_scores_across_folds.csv", index=False)

# Display the AUC DataFrame
auc_df


  0%|          | 0/3 [00:00<?, ?it/s]/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


43249
43249
{0: 0.7695825474216165, 1: 1.4273597359735974}
scale_pos_weight:  0.5391650948432328
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters: {'scale_pos_weight': 0.5391650948432328, 'n_jobs': -1, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.3}


 33%|███▎      | 1/3 [00:08<00:17,  8.93s/it]/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


44699
44699
{0: 0.7635894632546381, 1: 1.4484445884640311}
scale_pos_weight:  0.527178926509276
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters: {'scale_pos_weight': 0.527178926509276, 'n_jobs': -1, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.3}


 67%|██████▋   | 2/3 [00:17<00:08,  8.64s/it]/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


...


 96%|█████████▌| 201/209 [00:02<00:00, 71.18it/s]

0.625
0.11111111111111116
0.5377777777777777
0.6833333333333333
0.33333333333333337
0.57981220657277
0.7689393939393939
0.3636363636363637
0.367965367965368
0.6079545454545454
0.25
1.0


100%|██████████| 209/209 [00:02<00:00, 82.54it/s]
3it [00:08,  2.79s/it]

0.5588235294117647
0.5856774078689693


,Fold,Task,AUC
0,1,1,0.634921
1,1,2,0.295918
2,1,4,0.307692
3,1,5,0.566667
4,1,6,0.402174
...,...,...,...
613,3,205,0.607955
614,3,206,0.250000
615,3,207,1.000000
616,3,208,0.558824


In [125]:
mean_auc = auc_df.groupby("Task").mean().reset_index(drop=False)
mean_auc

,Task,AUC
0,1,0.461357
1,10,0.455556
2,100,0.939275
3,101,0.925217
4,102,0.869755
...,...,...
204,95,0.939078
205,96,0.918182
206,97,0.935185
207,98,0.936182


In [126]:
len(mean_auc[mean_auc["AUC"]==1])


0

In [127]:
len(mean_auc[mean_auc["AUC"]>0.90])

53

In [128]:
len(mean_auc[mean_auc["AUC"]>0.80])

68

In [129]:
len(mean_auc[mean_auc["AUC"]>0.70])

89

In [130]:
len(mean_auc[mean_auc["AUC"]>0.60])

115

In [131]:
len(mean_auc[mean_auc["AUC"]>0.50])

157

In [132]:
mean_auc.mean()

Task        inf
AUC     0.66309
dtype: float64

In [133]:
mean_auc.std()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


AUC    0.216629
dtype: float64

In [134]:
mean_auc.to_csv("scaffold_mean_AUCSeal.csv")

In [135]:
median_auc = auc_df.groupby("Task").median().reset_index(drop=False)
median_auc.to_csv("scaffold_median_AUCSeal.csv")

In [136]:
len(median_auc[median_auc["AUC"]==1])

1

In [137]:
len(median_auc[median_auc["AUC"]>0.90])

57

In [138]:
len(median_auc[median_auc["AUC"]>0.70])

88

In [139]:
len(median_auc[median_auc["AUC"]>0.50])

152